In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))

In [2]:
from multiversx_sdk_core import Address
from multiversx_sdk_core.transaction_builders import ContractCallBuilder, ESDTNFTTransferBuilder
from multiversx_sdk_core.transaction_builders import DefaultTransactionBuildersConfiguration
from multiversx_sdk_core import TokenPayment
from multiversx_sdk_network_providers import ProxyNetworkProvider
from tools.notebooks import env

config = DefaultTransactionBuildersConfiguration(chain_id=env.CHAIN_ID)
proxy = ProxyNetworkProvider(env.PROXY_URL)

In [ ]:
contract_address = env.ESCROW_CONTRACT
XMEX = env.XMEX
LKMEX = env.LKMEX
WXMEX = env.WXMEX

In [ ]:
from multiversx_sdk_core import AccountNonceHolder
from multiversx_sdk_wallet import UserSigner
from pathlib import Path

user = env.USER1
receiver = env.USER2
admin = env.ADMIN
owner = env.OWNER

user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))
owner_signer = UserSigner.from_pem_file(Path(env.OWNER_PEM))

lockFunds

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

transfers = [
    TokenPayment.meta_esdt_from_amount(XMEX, 1, 1000000, 18),
    TokenPayment.meta_esdt_from_amount(XMEX, 46, 1000000, 18)
]

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="lockFunds",
    caller=user,
    call_arguments=[receiver],
    gas_limit=30000000,
    esdt_transfers=transfers
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

withdraw receiver

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(env.USER2).nonce)

builder = ContractCallBuilder(
    config,
    contract=env.ESCROW_CONTRACT,
    function_name="withdraw",
    caller=env.USER2,
    call_arguments=[env.FORWARDER_CONTRACT],
    gas_limit=30000000
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

withdraw self

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="withdraw",
    caller=user,
    call_arguments=[user],
    gas_limit=30000000
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

withdraw - others (fail)

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(admin).nonce)

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="withdraw",
    caller=admin,
    call_arguments=[user],
    gas_limit=30000000
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

cancel transfer - user (fail)

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="cancelTransfer",
    caller=user,
    call_arguments=[user, receiver],
    gas_limit=30000000
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

addAdmin - owner (pass)

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(owner).nonce)

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="addAdmin",
    caller=owner,
    call_arguments=[admin],
    gas_limit=30000000
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

addAdmin - others (fail)

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="addAdmin",
    caller=user,
    call_arguments=[receiver],
    gas_limit=30000000
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

cancel transfer

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(env.ADMIN).nonce)

builder = ContractCallBuilder(
    config,
    contract=env.ESCROW_CONTRACT,
    function_name="cancelTransfer",
    caller=admin,
    call_arguments=[env.USER1, env.FORWARDER_CONTRACT],
    gas_limit=30000000
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

Token transfer

In [ ]:
transfer = TokenPayment.meta_esdt_from_amount(WXMEX, 2, 2000000, 18)

user = env.USER1
receiver = env.USER2
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

builder = ESDTNFTTransferBuilder(
    config,
    destination=receiver,
    sender=user,
    gas_limit=30000000,
    payment=transfer
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)